In [1]:
# -- what kinds of data are supported
# -- how to structure your data
# -- mock data and how to create some
# -- how much data you can store (Note: k8s requires blob storage configuration)
# -- adding metadata and uploading
# -- how to change the data later